# Visualizing Geospatial Data in Python

## By Paco Nathan posted Sun May 17, 2020 11:25 PM
## https://community.ibm.com/community/user/datascience/blogs/paco-nathan/2020/05/17/viz-geo-data-py     
 
Throughout the global pandemic, many people have spent lots of time viewing maps that visualize data. Important data. People who work in data science are probably seeing increased needs to work with geospatial data, especially for visualizations. There are increased needs to understand metrics about geographic regions, to analyze supply chain, make plans that take into account local conditions and rules, etc.

This article shows how to use two popular geospatial libraries in Python:

**geopandas:** extends Pandas to allow spatial operations on geometric types

**geoplot:** a high-level geospatial plotting library


The second library is especially helpful since it builds on top of several other popular geospatial libraries, to simplify the coding that’s typically required. Those include: cartopy, which in turn leverages Cython, NumPy, GEOS, Shapely, pyshp, PROJ, Six, and perhaps a few others such as mapclassify, depending on which features you need to use.

Note: all of this code is available in a Jupyter notebook at https://github.com/DerwenAI/ibm_dsc_articles/blob/master/2020_05/tutorial.ipynb


## Installation
You need two packages to run this tutorial: geopandas and geoplot
Installation should be quick. Just use the following command lines:

*#Run the following in terminal:*

conda create --name geoplot_compatible python=3.8

conda activate geoplot_compatible

conda install mamba -c conda-forge/label/mamba-alpha -c conda-forge

pip install Cython > 0.29.16

pip install descartes > 1.1.0

pip install imageio > 2.8.0

pip install jupyter > 1.0.0

pip install jupyterlab > 2.1.0

pip install matplotlib > 3.2.1

pip install shapely --no-binary shapely 

conda install -c anaconda git

pip install git+https://github.com/SciTools/cartopy.git --no-binary cartopy

pip install pysal --no-binary pysal

conda install --channel conda-forge cartopy

pip install geopandas

pip install geoplot

python -m ipykernel install --user --name=geoplot_compatible



To run this notebook in "geoplot_compatible", click "Kernel">"Change kernel" > "geoplot_compatible"

If you don't see "geoplot_compatible" as an option, refresh your browser page.

**Note:** if you run into problems with these installations, there are alternative approaches available. Some additional notes discuss how to build these dependencies on Linux  https://github.com/DerwenAI/ibm_dsc_articles/blob/master/2020_05/INSTALL.md

In [ ]:
#To import the required packages for Python:

import geoplot as gplt
import geopandas as gpd
import geoplot.crs as gcrs
import imageio
import pandas as pd
import pathlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import mapclassify as mc
import numpy as np

%matplotlib inline

pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

Note: if you run into problems with these installations, there are alternative approaches available. Some additional notes discuss how to build these dependencies on Linux  github.com/DerwenAI/ibm_dsc_articles/blob/master/2020_05/INSTALL.md

Terminology
Part of the learning curve for working with geospatial data is that there’s lots of special terminology used. Here’s a handy cheat-sheet for terms that you’re likely to encounter:

* **shapefile:** data file format used to represent items on a map
* **geometry:** a vector (generally a column in a dataframe) used to represent points, polygons, and other geometric shapes or locations, usually represented as well-known text (WKT)
* **polygon:** an area
* **point:** a specific location
* **basemap:** the background setting for a map, such as county borders in California
* **projection:** since the Earth is a 3D spheroid, chose a method for how an area gets flattened into 2D map, using some coordinate reference system (CRS)
* **colormap:** choice of a color palette for rendering data, selected with the cmap parameter
* **overplotting:** stacking several different plots on top of one another
* **choropleth:** using different hues to color polygons, as a way to represent data levels
* **kernel density estimation:** a data smoothing technique (KDE) that creates contours of shading to represent data levels
* **cartogram:** warping the relative area of polygons to represent data levels
* **quantiles:** binning data values into a specified number of equal-sized groups
* **voronoi diagram:** dividing an area into polygons such that each polygon contains exactly one generating point and every point in a given polygon is closer to its generating point than to any other; also called a Dirichlet tessellation

Okay, with those terms defined here for reference … let’s go!

### Load Some Data
We need to get some data to use for these examples. While geoplot includes plenty of sample datasets in the geojson format, it helps to know how to load your own data.

First, let’s get a shapefile from the US Census Bureau TIGER database to visualize state boundaries, which we’ll place into a maps subdirectory:

In [ ]:
usa = gpd.read_file("./maps/cb_2018_us_state_20m.shp")
usa.head()

Notice the geometry column above, which specifies polygon shapes.

Now we'll load the US Census data as a pandas dataframe and view a portion of it:

In [ ]:
state_pop = pd.read_csv("./maps/nst-est2018-alldata.csv")
state_pop.head()

Next we *merge* the shapefile with population data, joining on the state names:

In [ ]:
pop_states = usa.merge(state_pop, left_on="NAME", right_on="NAME")
pop_states.head()

Great, now that data is ready to plot a shape. We'll specify "California" by name:

In [ ]:
pop_states[pop_states.NAME=="California"].plot()

Alternatively, we can create a GeoDataFrame (a dataframe with geospatial data) by loading one of the sample datasets from geoplot, in this case the polygones for state boundaries. Search around in "gplt.datasets" to learn more about baselayers available for plotting without download:

In [ ]:
path = gplt.datasets.get_path("contiguous_usa")
contiguous_usa = gpd.read_file(path)
contiguous_usa.head()

Then plot the map of the US states:

In [ ]:
gplt.polyplot(contiguous_usa)

Then plot the locations of each city in the continental US as points (this data is also available through geoplot):

In [ ]:
path = gplt.datasets.get_path("usa_cities")
usa_cities = gpd.read_file(path)
usa_cities.head()

In [ ]:
continental_usa_cities = usa_cities.query('STATE not in ["HI", "AK", "PR"]')
gplt.pointplot(continental_usa_cities)

Composing those two, we'll use overplotting to show the cities and states in the continental US. Note how the ax variable for the state polygons provides an axis on which to plot the cities:

In [ ]:
ax = gplt.polyplot(contiguous_usa)
gplt.pointplot(continental_usa_cities, ax=ax)

That looks a bit stretched, so let's adjust the projection to use an Albers equal-area conic projection:

In [ ]:
ax = gplt.polyplot(contiguous_usa, projection=gcrs.AlbersEqualArea())
gplt.pointplot(continental_usa_cities, ax=ax)

## Representing Data
Now let's compare several different ways to visualize geospatial data. First, we'll change the hue of a city's plotted point based on that city's elevation, and also add a legend for people to decode the meaning of the different hues. The parameter lists start to get long-ish, so we'll split each parameter into a different line:

In [ ]:
ax = gplt.polyplot(contiguous_usa, projection=gcrs.AlbersEqualArea())

gplt.pointplot(
    continental_usa_cities,
    ax=ax,
    hue="ELEV_IN_FT",
    legend=True
)

We can also use the scale of each plotted point to represent another dimension. In this case, the scale of the city points is based on their elevation:

In [ ]:

ax = gplt.polyplot(
    contiguous_usa, 
    edgecolor="white",
    facecolor="lightgray",
    figsize=(12, 8),
    projection=gcrs.AlbersEqualArea()
)

gplt.pointplot(
    continental_usa_cities,
    ax=ax,
    hue="ELEV_IN_FT",
    cmap="Blues",
    scheme="quantiles",
    scale="ELEV_IN_FT",
    limits=(1, 10),
    legend=True,
    legend_var="scale",
    legend_kwargs={"frameon": False},
    legend_values=[-110, 1750, 3600, 5500, 7400],
    legend_labels=["-110 feet", "1750 feet", "3600 feet", "5500 feet", "7400 feet"]
)

ax.set_title("Cities in the continental US, by elevation", fontsize=16)

In [ ]:
?continental_usa_cities

## TASK 1: Plot the cities in the United States with a point scale for population and a color scale for elevation. 

With a choropleth we use different hues to shade polygons, to represent a dimension of data.

In [ ]:
ax = gplt.polyplot(contiguous_usa, projection=gcrs.AlbersEqualArea())

gplt.choropleth(
    contiguous_usa,
    hue="population",
    edgecolor="white",
    linewidth=1,
    cmap="Greens",
    legend=True,
    scheme="FisherJenks",
    legend_labels=[
        "<3 million", "3-6.7 million", "6.7-12.8 million",
        "12.8-25 million", "25-37 million"
    ],
    projection=gcrs.AlbersEqualArea(),
    ax=ax
)

A data smoothing technique known as kernel density estimation (KDE) creates contours to represent a dimension of data. In this case, we'll zoom in to view the traffic collisions in the NYC boroughs:

In [ ]:
boroughs = gpd.read_file(gplt.datasets.get_path("nyc_boroughs"))
collisions = gpd.read_file(gplt.datasets.get_path("nyc_collision_factors"))

ax = gplt.polyplot(boroughs, projection=gcrs.AlbersEqualArea())
gplt.kdeplot(collisions, cmap="Reds", shade=True, clip=boroughs, ax=ax)

Let's zoom out to try KDE on major population centers throughout the US:

In [ ]:
ax = gplt.polyplot(contiguous_usa, projection=gcrs.AlbersEqualArea())

gplt.kdeplot(
    continental_usa_cities, 
    cmap="Reds", 
    shade=True, 
    clip=contiguous_usa, 
    ax=ax
)

## TASK 2. Answer: Are we visualizing raw data in the above KDE plots? Why or why not (be specific)?

This next section shows how to work with data associated with areas (polygons). We'll load a dataset about obesity rates by US state:

In [ ]:
obesity = pd.read_csv(gplt.datasets.get_path("obesity_by_state"), sep="\t")
obesity.head()

Convert that into a GeoDataFrame using a join. Note how this adds the required "geometry" column:

In [ ]:
geo_obesity = contiguous_usa.set_index("state").join(obesity.set_index("State"))
geo_obesity.head()

Now we can use this data to plot a cartogram, which grows or shrinks polygons to represent a dimension of data – in this case, the obesity rates per state:

In [ ]:
gplt.cartogram(
    geo_obesity,
    scale="Percent",
    projection=gcrs.AlbersEqualArea()
)

One good approach to simplifying data visualization is binning the data into quantiles. These are equal-sized groups, in this case 10 quantiles for elevation:

In [ ]:
scheme = mc.Quantiles(continental_usa_cities["ELEV_IN_FT"], k=10)
scheme

Here we've divided the elevations into 10 quantiles with approximately 375 values each. Now let's assign a different hue to each quantile, plus a legend to explain them:

In [ ]:
gplt.pointplot(
    continental_usa_cities,
    projection=gcrs.AlbersEqualArea(),
    hue="ELEV_IN_FT",
    scheme=scheme,
    cmap="inferno_r",
    legend=True
)

Note how the colormap was changed to inferno_r. Next, let's add a filter for typical warnings that can be ignored:

In [ ]:
import warnings

warnings.filterwarnings("ignore", "GeoSeries.isna", UserWarning)

The next example uses a voronoi diagram, to calculate polygon areas based on a dimension of the data. Each polygon is centered on a generating point, such that every location in the polygon is closer to its generating point than to any other.

In the following example, we'll plot the locations primary schools in Melbourne, Australia, and use a voronoi diagram to show where they are concentrated:

In [ ]:
melbourne = gpd.read_file(gplt.datasets.get_path("melbourne"))
df = gpd.read_file(gplt.datasets.get_path("melbourne_schools"))
melbourne_primary_schools = df.query('School_Type == "Primary"')

ax = gplt.voronoi(
    melbourne_primary_schools,
    clip=melbourne,
    linewidth=0.5,
    edgecolor="white",
    projection=gcrs.Mercator()
)

gplt.polyplot(
    melbourne, 
    edgecolor="None", 
    facecolor="lightgray",
    ax=ax
)

gplt.pointplot(
    melbourne_primary_schools,
    color="black",
    ax=ax,
    s=1,
    extent=melbourne.total_bounds
)

plt.title("Primary Schools in Greater Melbourne, 2018")

Let's construct a voronoi diagram for the elevations of US cities. This is a data smoothing technique since the elevations are for points, but we'll "spread" those values across areas:

In [ ]:
proj = gplt.crs.AlbersEqualArea(
    central_longitude=-98,
    central_latitude=39.5
)

ax = gplt.voronoi(
    continental_usa_cities,
    hue="ELEV_IN_FT",
    clip=contiguous_usa,
    projection=proj,
    cmap="Reds",
    legend=True,
    edgecolor="white",
    linewidth=0.01
)

gplt.polyplot(
    contiguous_usa,
    ax=ax,
    extent=contiguous_usa.total_bounds,
    edgecolor="black",
    linewidth=1,
    zorder=1
)

## TASK 3: Fix this error message so you can plot your data.